In [1]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
# import seaborn as sns
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")




/homes/metogra/iufarias/xroms/xroms/interp.py:18: UserWarning: xESMF is not installed, so `interpll` will not run.
  warnings.warn("xESMF is not installed, so `interpll` will not run.")
/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
client = Client(n_workers=80)
# client = Client(cluster)

In [4]:
client

<Client: 'tcp://127.0.0.1:42248' processes=80 threads=160, memory=503.40 GiB>

## Reading CROCO (3D)

In [61]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
                        'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}


path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*2012-06*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-06*.nc',data_vars='minimal',chunks=chunks)

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)



Converting to xroms

In [ ]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [ ]:

# plt.rcParams['text.usetex'] = True
# plt.rcParams.update({'font.size': 22})

# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = ['Lucida Grande']

In [54]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

#### Spectral parameters

In [55]:
dx=0.5
nf=2
wdws='hann' #'flattop','hann'
wdw_cor=True
scl='density'

# $N^2_r$
JAN=2.275e-06
FEB=4.178e-06
MAR=2.116e-05
APR=2.863e-05
MAY=1.790e-05
JUN=5.648e-05
AVG=2.175e-05

In [56]:
rho_full=df.rho

In [57]:
z=rho_full.z_rho0.compute()

In [58]:
%%time

N2=[]
for ixx in range(rho_full.shape[0]):
    N2.append(df.xroms.N2[:,:,120:-120,120:800][ixx,-2].compute().mean())

CPU times: user 3min 36s, sys: 1min 8s, total: 4min 45s
Wall time: 13min


In [59]:
np.mean(N2)

5.648094147306015e-05

#### EKE (FULL)

In [24]:
u_full_psi=df.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_full_psi=df.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

# u_full_psi=df.u.xroms.to_grid('psi', 's_rho')[0,:,120:-120,120:800]
# v_full_psi=df.v.xroms.to_grid('psi', 's_rho')[0,:,120:-120,120:800]


In [43]:
V_full_interp=np.zeros([v_full_psi.shape[0],10,v_full_psi.shape[2],v_full_psi.shape[3]])*np.nan;

In [44]:
%%time
# v_full_interp=v_full_psi.xroms.isoslice(np.linspace(-5,-50, 10)).compute()
for ix in range(u_full_psi.shape[0]):
    V_full_interp[ix]=v_full_psi[ix].xroms.isoslice(np.linspace(-5,-50, 10)).compute()

CPU times: user 3min 39s, sys: 1min 30s, total: 5min 10s
Wall time: 7min 15s


In [48]:
# xr.DataArray(V_full_interp)

In [49]:
%%time

Vf=xrft.isotropic_power_spectrum(xr.DataArray(V_full_interp),
                                  dim=['dim_2','dim_3'],
                          nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                 window=wdws,window_correction=wdw_cor)

# with ProgressBar():
#     Vf=xrft.isotropic_power_spectrum(V_full_interp,
#                                       dim=['eta_v','xi_u'],
#                               nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
#                                      window=wdws,window_correction=wdw_cor)

CPU times: user 19min 32s, sys: 1h 6min 57s, total: 1h 26min 30s
Wall time: 6min 9s


In [69]:
U_full_interp=np.zeros([u_full_psi.shape[0],10,u_full_psi.shape[2],u_full_psi.shape[3]])*np.nan;

In [2]:
# %%time
# # u_full_interp=u_full_psi.xroms.isoslice(np.linspace(-5,-50, 10)).compute()

# for ix in range(u_full_psi.shape[0]):
#     U_full_interp[ix]=u_full_psi[ix].xroms.isoslice(np.linspace(-5,-50, 10)).compute()


In [72]:
%%time


Uf=xrft.isotropic_power_spectrum(xr.DataArray(U_full_interp),
                                  dim=['dim_2','dim_3'],
                          nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                 window=wdws,window_correction=wdw_cor)


# with ProgressBar():
    # Uf=xrft.isotropic_power_spectrum(u_full_interp,
    #                                   dim=['eta_v','xi_u'],
    #                           nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
    #                                  window=wdws,window_correction=wdw_cor)

CPU times: user 18min 54s, sys: 1h 4min 18s, total: 1h 23min 13s
Wall time: 5min 44s


In [83]:
Eke_full=0.5*(Vf**2+Uf**2)
Eke_full=Eke_full.rename(dim_0='ocean_time',dim_1='z_rho_psi')

In [84]:
%%time
Eke_full.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/Eke_full_04.nc')

CPU times: user 57.7 ms, sys: 35.6 ms, total: 93.3 ms
Wall time: 213 ms


#### EKE (SMTH)

In [14]:
v_smth_psi=dt.v[:,:,120:-120,120:800]
u_smth_psi=dt.u[:,:,120:-120,120:800]

# v_smth_psi=dt.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
# u_smth_psi=dt.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]



In [1]:
u_smth_interp=u_smth_psi[0].xroms.isoslice(np.linspace(-5,-50, 10)).compute()

In [13]:
%%time

with ProgressBar():
    Ut=xrft.isotropic_power_spectrum(u_smth_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).compute()

with ProgressBar():
    Vt=xrft.isotropic_power_spectrum(v_smth_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).compute()

In [14]:
Eke_smth=(0.5*(Ut**2+Vt**2))

In [29]:
Eke_smth.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/Eke_smth_02.nc')

#### EPE (FULL)

In [10]:
rho_full_psi=df.rho.xroms.to_grid('psi', 's_rho')
rho_full_interp=rho_full_psi.xroms.isoslice(np.linspace(-5,-50, 10))[:,:,120:-120,120:800]


In [11]:
b_full_interp=xroms.buoyancy(rho_full_interp)
# rho_full_interp=xroms.potential_density(t_full_interp,s_full_interp)
# bf=xroms.buoyancy(rho_full_interp-1025)

In [32]:
%%time
with ProgressBar():
    Bf=xrft.isotropic_power_spectrum(b_full_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).compute()

CPU times: user 1min 30s, sys: 48.8 s, total: 2min 19s
Wall time: 11min 33s


In [17]:
N2r=(5.2*10**-3)**2
# N2r=(3.9e-3)**2

In [34]:
Epe_full=(1/(2*N2r))*Bf

In [35]:
Epe_full.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/Epe_full_02.nc')

#### EPE (SMTH)

In [12]:

rho_smth_psi=dt.rho.xroms.to_grid('psi', 's_rho')
rho_smth_interp=rho_smth_psi.xroms.isoslice(np.linspace(-5,-50, 10))[:,:,120:-120,120:800]


In [13]:
b_smth_interp=xroms.buoyancy(rho_smth_interp)

In [38]:
%%time
with ProgressBar():
    Bt=xrft.isotropic_power_spectrum(b_smth_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).compute()

CPU times: user 1min 28s, sys: 40.5 s, total: 2min 9s
Wall time: 11min 11s


In [39]:
Epe_smth=(1/(2*N2r))*Bt

In [40]:
Epe_smth.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/Epe_smth.nc')

#### w'b' (FULL)

In [17]:
w_full_psi=df.w.xroms.to_grid('psi', 's_rho')
w_full_interp=w_full_psi.xroms.isoslice(np.linspace(-5,-50, 10))[:,:,120:-120,120:800]



In [20]:
%%time

with ProgressBar():
    wb_f=xrft.isotropic_cross_spectrum(w_full_interp,b_full_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real.compute()


CPU times: user 3min 3s, sys: 1min 21s, total: 4min 24s
Wall time: 27min 16s


#### w'b' (SMTH)

In [21]:
w_smth_psi=dt.w.xroms.to_grid('psi', 's_rho')
w_smth_interp=w_smth_psi.xroms.isoslice(np.linspace(-5,-50, 10))[:,:,120:-120,120:800]


In [2]:
%%time

with ProgressBar():
    wb_t=xrft.isotropic_cross_spectrum(w_smth_interp,b_smth_interp,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real.compute()

NameError: name 'ProgressBar' is not defined

In [23]:
wb_f.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/wb_full.nc')
wb_t.to_netcdf('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/wb_smth.nc')

In [82]:
# AA=xr.open_dataarray('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/wb_full_01.nc')
# BB=xr.open_dataarray('/homes/metogra/iufarias/Documents/data/spec/100_int/KE/wb_smth_01.nc')

In [251]:
# fig,ax=plt.subplots(2,1,figsize=(9,6))


# im1=ax[0].pcolormesh(AA.freq_r/dx,AA.z_rho_psi,AA.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[0].set_xscale('log')
# ax[0].set_xlim(1e-2,0)
# ax[0].set_title(r'(a)',color='k',fontsize=20)
# # ax[0].set_ylabel('Depth [m]',fontsize=20,x=0.1)



# im2=ax[1].pcolormesh(BB.freq_r/dx,BB.z_rho_psi,BB.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[1].set_xscale('log')
# ax[1].set_xlim(1e-2,0)
# ax[1].set_title(r'(b)',fontsize=20)
# # ax[1].set_ylabel('Depth [m]',fontsize=20,x=0.1)
# fig.supylabel('Depth [m]',fontsize=20)
# fig.supxlabel(r'Wavenumber [km$^{-1}$]',fontweight='bold',fontsize=20,y=-0.00001)
    

    
# fig.subplots_adjust(right=0.8,hspace=0.4)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# cbar=fig.colorbar(im, cax=cbar_ax)
# # cbar=fig.colorbar(im,cax=ax[0]
# cbar.set_label(r'Spectral Density [m$^3\,$s$^{-3}$]',labelpad=12)

# # plt.tight_layout()
# plt.savefig('/homes/metogra/iufarias/Documents/figures/2_CROCO/CROCO_zintegrated/z_int/wb_z.png')

# $N_r^2$

In [44]:
n2_full_t

<xarray.DataArray 'N2' (ocean_time: 120, eta_v: 1519, xi_u: 999)>
dask.array<getitem, shape=(120, 1519, 999), dtype=float64, chunksize=(3, 1519, 999), chunktype=numpy.ndarray>
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2012-04-01T06:00:00 ... 2012-05-01
    s_w         float32 -1.0
  * eta_v       (eta_v) int64 0 1 2 3 4 5 6 ... 1513 1514 1515 1516 1517 1518
  * xi_u        (xi_u) int64 0 1 2 3 4 5 6 7 ... 991 992 993 994 995 996 997 998
Attributes:
    long_name:           buoyancy frequency squared, or vertical buoyancy gra...
    units:               1/s^2
    online_operation:    instant
    interval_operation:  6 h
    interval_write:      6 h
    cell_methods:        time: point
    grid:                <xgcm.Grid>\nX Axis (not periodic, boundary=None):\n...
    name:                N2

In [18]:
%%time
n2_full_t=xroms.N2(rho_smth_psi, rho_smth_psi.attrs['grid'])[:,0,:,:].compute()
# n2_full_t=np.gradient(b_full_interp,axis=1)

CPU times: user 15 µs, sys: 9 µs, total: 24 µs
Wall time: 45.1 µs


In [19]:
%%time
n2_smth_t=xroms.N2(rho_smth_psi, rho_smth_psi.attrs['grid'])[:,0,:,:].compute()
# n2_smth_t=np.gradient(b_smth_interp,axis=1)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 13.1 µs


#### b and w specta (FULL and SMTH)

In [73]:
# %%time
# with ProgressBar():

#     Wf=xrft.isotropic_power_spectrum(w_full_interp),
#                                       dim=['eta_v','xi_u'],
#                               nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
#                                      window=wdws,window_correction=wdw_cor)

#     Wt=xrft.isotropic_power_spectrum(w_smth_interp),
#                                       dim=['eta_v','xi_u'],
#                               nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
#                                      window=wdws,window_correction=wdw_cor)